In [2]:
import pandas as pd

In [84]:
data=pd.read_csv('/Users/siddharth/Downloads/study/entity-annotated-corpus/ner_dataset.csv',encoding='latin1').fillna(method="ffill")

In [85]:
data

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,Sentence: 1,of,IN,O
2,Sentence: 1,demonstrators,NNS,O
3,Sentence: 1,have,VBP,O
4,Sentence: 1,marched,VBN,O
...,...,...,...,...
1048570,Sentence: 47959,they,PRP,O
1048571,Sentence: 47959,responded,VBD,O
1048572,Sentence: 47959,to,TO,O
1048573,Sentence: 47959,the,DT,O


In [40]:
fileName="/Volumes/Data/token_pos_tagged - token_pos_tagged (1).tsv"
df = pd.read_csv(fileName, sep='\t', header=None,names=['Word','POS','Tag'])

In [79]:
data

,Word,POS,Tag,Sentence #
0,1,CD,TYPE,Sentence:1
1,tsp,NN,UNIT,Sentence:1
2,powdered,VBD,NAME,Sentence:1
3,gelatine,NN,NAME,Sentence:1
4,,,,Sentence:1
...,...,...,...,...
110416,taste,VB,COM,Sentence:18669
110417,,,,Sentence:18669
110418,250ml,CD,QTY,Sentence:18670
110419,thick,JJ,COM,Sentence:18670


In [48]:
df1 = df.replace(np.nan, '', regex=True)


In [66]:
sentence=[]
count=1
for i in df1.Word:
    sentence.append('Sentence:'+str(count))
    if len(i)==0:
        count+=1
        
    


In [88]:
data=df1

In [89]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [226]:
word1 = list(set(data["Word"].values))

In [211]:
words.append("ENDPAD")

In [221]:
getter = SentenceGetter(data)

In [223]:
sent = getter.get_next()

In [224]:
sent

In [225]:
words = list(set(data["Word"].values))
words.append("ENDPAD")
n_words = len(words); n_words
tags = list(set(data["Tag"].values))
n_tags = len(tags); n_tags
class SentenceGetter(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
getter = SentenceGetter(data)
sent = getter.get_next()
sentences = getter.sentences
max_len = 75
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)
y = [[tag2idx[w[2]] for w in s] for s in sentences]



In [238]:
X[0].shape

(75,)

In [101]:
tag2idx.update({'O':0})

In [102]:
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
from keras.utils import to_categorical
y = [to_categorical(i, num_classes=n_tags) for i in y]
from sklearn.model_selection import train_test_split
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [348]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = Dense(50, activation="relu")(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output
model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])


/opt/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:346: UserWarning: CRF.loss_function is deprecated and it might be removed in the future. Please use losses.crf_loss instead.
  warnings.warn('CRF.loss_function is deprecated '
/opt/anaconda3/lib/python3.7/site-packages/keras_contrib/layers/crf.py:353: UserWarning: CRF.accuracy is deprecated and it might be removed in the future. Please use metrics.crf_accuracy
  warnings.warn('CRF.accuracy is deprecated and it '


In [381]:

input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len)(input)  # 20-dim embedding
model=Conv1D(filters=32, kernel_size=5,padding='causal')(model)
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = Dense(50, activation="relu")(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output
model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])



In [382]:
model.summary()

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_32 (InputLayer)        (None, 75)                0         
_________________________________________________________________
embedding_33 (Embedding)     (None, 75, 20)            74980     
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 75, 32)            3232      
_________________________________________________________________
bidirectional_29 (Bidirectio (None, 75, 100)           33200     
_________________________________________________________________
dense_61 (Dense)             (None, 75, 50)            5050      
_________________________________________________________________
crf_26 (CRF)                 (None, 75, 6)             354       
Total params: 116,816
Trainable params: 116,816
Non-trainable params: 0
____________________________________________________

In [378]:
model.summary()

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 75)                0         
_________________________________________________________________
embedding_30 (Embedding)     (None, 75, 20)            74980     
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 75, 32)            3232      
_________________________________________________________________
bidirectional_27 (Bidirectio (None, 75, 100)           33200     
_________________________________________________________________
dense_59 (Dense)             (None, 75, 50)            5050      
_________________________________________________________________
crf_24 (CRF)                 (None, 75, 6)             354       
Total params: 116,816
Trainable params: 116,816
Non-trainable params: 0
____________________________________________________

In [375]:
model.summary()

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        (None, 75)                0         
_________________________________________________________________
embedding_29 (Embedding)     (None, 75, 20)            74980     
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 71, 32)            3232      
_________________________________________________________________
bidirectional_26 (Bidirectio (None, 71, 100)           33200     
_________________________________________________________________
dense_58 (Dense)             (None, 71, 50)            5050      
_________________________________________________________________
crf_23 (CRF)                 (None, 71, 6)             354       
Total params: 116,816
Trainable params: 116,816
Non-trainable params: 0
____________________________________________________

In [383]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=5,
                    validation_split=0.1, verbose=1)

/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 15122 samples, validate on 1681 samples
Epoch 1/5
15122/15122 [==============================] - 45s 3ms/step - loss: 0.0682 - crf_viterbi_accuracy: 0.9750 - val_loss: 0.0067 - val_crf_viterbi_accuracy: 0.9970
Epoch 2/5
15122/15122 [==============================] - 44s 3ms/step - loss: -3.1191e-04 - crf_viterbi_accuracy: 0.9983 - val_loss: -0.0051 - val_crf_viterbi_accuracy: 0.9986
Epoch 3/5
15122/15122 [==============================] - 42s 3ms/step - loss: -0.0100 - crf_viterbi_accuracy: 0.9992 - val_loss: -0.0121 - val_crf_viterbi_accuracy: 0.9989
Epoch 4/5
15122/15122 [==============================] - 41s 3ms/step - loss: -0.0174 - crf_viterbi_accuracy: 0.9996 - val_loss: -0.0189 - val_crf_viterbi_accuracy: 0.9991
Epoch 5/5
15122/15122 [==============================] - 42s 3ms/step - loss: -0.0242 - crf_viterbi_accuracy: 0.9998 - val_loss: -0.0253 - val_crf_viterbi_accuracy: 0.9991


In [400]:
test_sentence = ["2 tablespoons cantaloupe for low carb / keto OR frozen mango chunks"]

In [401]:
for i in test_sentence:
    test_sentence=i.split()

In [402]:
test_sentence

['2',
 'tablespoons',
 'cantaloupe',
 'for',
 'low',
 'carb',
 '/',
 'keto',
 'OR',
 'frozen',
 'mango',
 'chunks']

In [403]:
x_test_sent = pad_sequences(sequences=[[word2idx.get(w, 0) for w in test_sentence]],
                            padding="post", value=0, maxlen=max_len)

In [404]:
p = model.predict(np.array([x_test_sent[0]]))
p = np.argmax(p, axis=-1)
print("{:15}||{}".format("Word", "Prediction"))
print(30 * "=")
for w, pred in zip(test_sentence, p[0]):
    print("{:15}: {:5}".format(w, tags[pred]))

Word           ||Prediction
2              : QTY  
tablespoons    : UNIT 
cantaloupe     :      
for            : COM  
low            : COM  
carb           : NAME 
/              : COM  
keto           :      
OR             :      
frozen         : COM  
mango          : NAME 
chunks         : UNIT 


In [405]:
import requests

url = 'https://www.w3schools.com/python/demopage.php'
myobj = {'somekey': 'somevalue'}

x = requests.post(url, data = myobj)

print(x.text)

In [406]:
175253+106393

281646

In [407]:
import pandas as pd

In [409]:
df=pd.read_csv('/Users/siddharth/Desktop/go-code/recipes_202005071734.csv')

In [410]:
df.shape

(175253, 1)

In [416]:
df.source_url

0         https://www.bakedbyanintrovert.com/buttery-jam...
1         https://www.bakedbyanintrovert.com/best-ever-b...
2         https://www.bakedbyanintrovert.com/chocolate-c...
3         https://www.bakedbyanintrovert.com/honey-nut-z...
4         https://www.bakedbyanintrovert.com/parmesan-bu...
                                ...                        
175248           https://apriljharris.com/beef-in-red-wine/
175249    https://yummyindiankitchen.com/chicken-dry-fry...
175250    https://www.bunsinmyoven.com/buffalo-chicken-b...
175251       https://dinnerthendessert.com/haluski-noodles/
175252    http://wishfulchef.com/cranberry-pecan-couscou...
Name: source_url, Length: 175253, dtype: object

In [414]:
from urllib.parse import urlparse

domain = urlparse('http://www.example.test/foo/bar').netloc
print(domain) # -

www.example.test


In [418]:
domain=[]
domain=[urlparse(i).netloc for i in df.source_url]

In [420]:
len(set(domain))

523